In [11]:
import random

### 输入：input_file:输入带宽文件路径
###       output_file:输出mahimahi trace路径
###      interval: 带宽文件的采样间隔，单位为 秒

def trace_gen(input_file, output_file, interval):
    PACKET_SIZE = 1500.0  # bytes
    BITS_IN_BYTE = 8.0
    MBITS_IN_BITS = 1024**2
    MILLISECONDS_IN_SECONDS = 1000.0

    band_list = []
    with open(input_file,'r') as f:
        band = f.readline()
        while(band):
            band_list.append(float(band.replace('\n', '').strip()))
            band = f.readline()
    mahimahi_trace = []
    
    ## 每个带宽值持续的period(ms)
    period_len = int(MILLISECONDS_IN_SECONDS * interval)
    period_start = 0
    for band in band_list:
        bitsPerPeriod = int(band*interval*(10**6))
        bytesPerPeriod = int(bitsPerPeriod/8)
        packetsPerPeriod = int(round(bytesPerPeriod/PACKET_SIZE))
        mahimahi_trace+=(sorted([random.randint(period_start,period_start+period_len-1) for _ in range(packetsPerPeriod)]))
        period_start+=period_len
        
    
    with open(output_file, "w+") as f:
        for d in mahimahi_trace:
            f.write(str(d)+'\n')
            
    print('生成成功！')
            

trace_gen('cycle_24m_12m.txt', './cycle_trace/cycle_24m_12m.log', 1)
        

生成成功！


In [92]:
%run trace_gen.py

生成成功！


In [90]:
a

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os

def getTrace(path, interval):
	PACKET_SIZE = 1500.0  # bytes
	BITS_IN_BYTE = 8.0
	MBITS_IN_BITS = 1024**2
	MILLISECONDS_IN_SECONDS = 1000.0 
	N = 100
	time_all = []
	packet_sent_all = []
	last_time_stamp = 0
	packet_sent = 0
	with open(path, 'rb') as f:
		for line in f:
			if(len(line.split())==0):
				continue
			time_stamp = int(line.split()[0])
			if time_stamp == last_time_stamp:
				packet_sent += 1
				continue
			else:
				time_all.append(last_time_stamp)
				packet_sent_all.append(packet_sent)
				packet_sent = 1
				last_time_stamp = time_stamp

		traces = []
		start_time = 0   # seconds
		count = 0
		pkt_sum = 0
		epoch_start_ts = time_all[0]
		for i in range(len(time_all)):
			if(time_all[i]>=start_time*MILLISECONDS_IN_SECONDS and time_all[i]<=(start_time+interval)*MILLISECONDS_IN_SECONDS):
				pkt_sum += packet_sent_all[i]
			else:
				start_time+=interval

				throughputInSec = ((PACKET_SIZE * BITS_IN_BYTE * pkt_sum) / (time_all[i]-epoch_start_ts))*MILLISECONDS_IN_SECONDS / MBITS_IN_BITS
				traces.append(throughputInSec)
				epoch_start_ts = time_all[i]
				pkt_sum = 0

	return traces


In [7]:
getTrace('./cycle_trace/cycle_24m_12m.log', 1)

[11.455524456012737,
 22.853851318359375,
 11.444091796875,
 22.853851318359375,
 11.455535888671875]

In [8]:
def getTrace(path):
	PACKET_SIZE = 1500.0  # bytes
	BITS_IN_BYTE = 8.0
	MBITS_IN_BITS = 1024**2
	MILLISECONDS_IN_SECONDS = 1000.0 
	N = 100
	time_all = []
	packet_sent_all = []
	last_time_stamp = 0
	packet_sent = 0
	with open(path, 'rb') as f:
		for line in f:
			if(len(line.split())==0):
				continue
			time_stamp = int(line.split()[0])
			if time_stamp == last_time_stamp:
				packet_sent += 1
				continue
			else:
				time_all.append(last_time_stamp)
				packet_sent_all.append(packet_sent)
				packet_sent = 1
				last_time_stamp = time_stamp

		traces = []
		start_time = 0   # seconds
		count = 0
		pkt_sum = 0
		epoch_start_ts = time_all[0]
		for i in range(len(time_all)):
			if(time_all[i]>=start_time*MILLISECONDS_IN_SECONDS and time_all[i]<=(start_time+1)*MILLISECONDS_IN_SECONDS):
				pkt_sum += packet_sent_all[i]
			else:
				start_time+=1

				throughputInSec = ((PACKET_SIZE * BITS_IN_BYTE * pkt_sum) / (time_all[i]-epoch_start_ts))*MILLISECONDS_IN_SECONDS / MBITS_IN_BITS
				traces.append(throughputInSec)
				epoch_start_ts = time_all[i]
				pkt_sum = 0

	return traces


In [9]:
getTrace('./cycle_trace/cycle_24m_12m.log')

[11.455524456012737,
 22.853851318359375,
 11.444091796875,
 22.853851318359375,
 11.455535888671875]

In [12]:
%run band_from_mahimahi_s.py